In [1]:
import numpy as np

from functions import *

In [2]:
a = np.array([1, 2, 3, 4, 5, 6, 7, 8 ])
print(a)
print(np.clip(a, a_min = 5, a_max= np.inf))

[1 2 3 4 5 6 7 8]
[5. 5. 5. 5. 5. 6. 7. 8.]


In [3]:
nx = 100
ny = 100
num_species = 3
nB = 15

f = np.random.random((num_species, 9, nx, ny))

phi = np.ones(num_species)
molecular_weight = np.ones(num_species)

In [8]:
def bgk_step(f):

    #################### Before streaming ####################

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    ux_star_s, uy_star_s = calculate_u_star(CHI_sc, rho_s, rho_mix, ux_s, uy_s)

    feq = equilibrium(f, rho_s, phi, ux_star_s, uy_star_s)

    g_dagger_s =calculate_g_dagger(f, feq, lambda_s)

    #################### After streaming ####################

    f_streamed = lattice_stream(f)

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f_streamed, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    Chi_S = calculate_Chi_S(CHI_sc, rho_s, rho_mix)



In [9]:
bgk_step(f)

In [10]:
b = np.array([
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    ]
)
b[1,:] *= 2
b[2,:] *= 3
print(b)

[[ 1  2  3  4  5  6  7  8]
 [ 2  4  6  8 10 12 14 16]
 [ 3  6  9 12 15 18 21 24]]


In [16]:
delta = a[None, :] -b
print(delta)

[[  0   0   0   0   0   0   0   0]
 [ -1  -2  -3  -4  -5  -6  -7  -8]
 [ -2  -4  -6  -8 -10 -12 -14 -16]]
